In [1]:
cd ..

/Users/amlvt225/Code/GitHub/donkey_golf


In [32]:
pd.options.display.max_columns=200

In [2]:
from inspect import getsource

import donkey_golf as dg

import donkey_golf.data_things as DT

import oyaml
import pandas as pd
import logging
import psycopg2

from sqlalchemy import create_engine

from donkey_golf import config

# create logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

/Users/amlvt225/Code/VirtualEnvs/venv_dgolf/lib/python3.7/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [3]:
gs = DT.GameScoreboard()

2019-05-16 13:51:59,285 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-16 13:51:59,296 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-16 13:51:59,558 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056552
2019-05-16 13:51:59,559 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [ ]:
gs.run()

In [ ]:
gs.data

In [ ]:
df = gs.data

In [ ]:
self.data['donkey_score'] = self.data['donkey_score'].astype(int)

self.data.loc[self.data['donkey_score'] > 0, 'donkey_score'] = "+" + self.data['donkey_score'].astype(str)

self.data.loc[self.data['donkey_score'] == 0, 'donkey_score'] = 'E'

In [ ]:
df

In [ ]:
df.loc[]

In [6]:
lb = DT.PullLeaderboard(user_id=6)

2019-05-16 13:52:15,461 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-16 13:52:15,474 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-16 13:52:15,739 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056552
2019-05-16 13:52:15,741 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [7]:
lb.tourney_status

'in_progress'

In [8]:
lb.run()

2019-05-16 13:52:16,035 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-16 13:52:16,349 - donkey_golf.data_things.tourney_leaderboard - INFO - Pulling users active tourney team
2019-05-16 13:52:16,350 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-16 13:52:16,644 - donkey_golf.data_things.tourney_leaderboard - INFO - Determing data refresh times


In [9]:
lb.data

,pos,player,to_par,today,thru,r1,r2,r3,r4,tot,tourney_id,donkey_score,missed_cut,player_left,team_count,load_date
0,1,Brooks Koepka,-7,-7,F,63,--,--,--,63,401056552,-7,0,0,11,2019-05-16 13:52:04.354189
1,2,Tommy Fleetwood,-3,-3,F,67,--,--,--,67,401056552,-3,0,0,3,2019-05-16 13:52:04.354189
2,T3,Mike Lorenzo-Vera,-2,-2,F,68,--,--,--,68,401056552,-2,0,0,0,2019-05-16 13:52:04.354189
3,T3,Chez Reavie,-2,-2,F,68,--,--,--,68,401056552,-2,0,0,0,2019-05-16 13:52:04.354189
4,T3,Luke List,-2,-2,F,68,--,--,--,68,401056552,-2,0,0,0,2019-05-16 13:52:04.354189
5,T3,Paul Casey,-2,-2,4,--,--,--,--,--,401056552,-2,0,1,3,2019-05-16 13:52:04.354189
6,T3,Tony Finau,-2,-2,4,--,--,--,--,--,401056552,-2,0,1,8,2019-05-16 13:52:04.354189
7,T8,Rickie Fowler,-1,-1,F,69,--,--,--,69,401056552,-1,0,0,10,2019-05-16 13:52:04.354189
8,T8,Matt Wallace,-1,-1,F,69,--,--,--,69,401056552,-1,0,0,0,2019-05-16 13:52:04.354189
9,T8,Danny Lee,-1,-1,6,--,--,--,--,--,401056552,-1,0,1,0,2019-05-16 13:52:04.354189


In [10]:
wha = DT.TeamPlayerResults()

2019-05-16 16:41:25,002 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-16 16:41:25,017 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-16 16:41:25,371 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056552
2019-05-16 16:41:25,373 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [11]:
wha.run()

2019-05-16 16:41:30,535 - donkey_golf.data_things.team_player_results - INFO - Pulling full team results by player
2019-05-16 16:41:30,537 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [26]:
team_player_df = wha.data

In [28]:
team_player_df = team_player_df.merge(df_sb, how='left', on='id',
              suffixes=('_team','_gs'))

In [35]:
team_player_df.sort_values('rank', inplace=True)

In [33]:
team_player_df.loc[team_player_df['username_team'] == 'Geo1630']

,id,username_team,pos,player,to_par,today,thru,r1,r2,r3,r4,tot,tourney_id,donkey_score_team,missed_cut_team,player_left_team,team_count,load_date,username_gs,pay_flag,donkey_score_gs,missed_cut_gs,player_left_gs,rank,final_rank
0,27,Geo1630,1,Brooks Koepka,-7,-7,F,63,--,--,--,63,401056552,-7,0,0,11,2019-05-16 20:40:04.435092,Geo1630,Y,-2,0,4,8,8
24,27,Geo1630,T7,Rickie Fowler,-1,-1,F,69,--,--,--,69,401056552,-1,0,0,10,2019-05-16 20:40:04.435092,Geo1630,Y,-2,0,4,8,8
36,27,Geo1630,T7,Tony Finau,-1,-1,13,--,--,--,--,--,401056552,-1,0,1,8,2019-05-16 20:40:04.435092,Geo1630,Y,-2,0,4,8,8
55,27,Geo1630,T16,Jon Rahm,E,E,11,--,--,--,--,--,401056552,0,0,1,6,2019-05-16 20:40:04.435092,Geo1630,Y,-2,0,4,8,8
56,27,Geo1630,T16,Dustin Johnson,E,E,11,--,--,--,--,--,401056552,0,0,1,4,2019-05-16 20:40:04.435092,Geo1630,Y,-2,0,4,8,8
62,27,Geo1630,T39,Phil Mickelson,+1,+1,10,--,--,--,--,--,401056552,1,0,1,3,2019-05-16 20:40:04.435092,Geo1630,Y,-2,0,4,8,8
105,27,Geo1630,T88,Kevin Kisner,+3,+3,10,--,--,--,--,--,401056552,3,0,1,3,2019-05-16 20:40:04.435092,Geo1630,Y,-2,0,4,8,8


In [22]:
for i, row in wha.data.iterrows():
    print(df_sb.loc[df_sb['id'] == row['id'], 'donkey_score'].values[0])
    #print(row['username'])

-2
+5
-5
-8
-3
+3
-11
-9
-9
+1
-6
-9
-6
-11
+5
+3
-8
+1
+1
+3
+3
-3
-5
+3
-2
-9
+9
+12
+9
-9
+8
-9
+5
-3
-6
-6
-2
-8
+3
-11
+3
-9
-8
-5
+8
-5
+8
-9
+1
+9
+12
-11
-11
+5
+5
-2
-2
+5
+3
+1
-5
+3
-2
-5
+3
+9
-8
+9
+1
-3
+12
+1
-9
-9
+5
-6
+8
-9
-6
+5
-11
-9
+5
-9
-5
+9
+5
+5
+12
-11
+3
-3
-3
+3
-9
+12
-8
-8
+8
+8
+3
-6
+8
+5
+9
-2
+3
+5
+12
-3
+5
+12


In [14]:
GS = DT.GameScoreboard()

2019-05-17 08:34:13,339 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-17 08:34:13,353 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-17 08:34:13,708 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056552
2019-05-17 08:34:13,709 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [15]:
GS.run()

2019-05-17 08:34:19,324 - donkey_golf.data_things.game_scoreboard - INFO - Pulling aggregated team score for scoreboard
2019-05-17 08:34:19,326 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-17 08:34:19,684 - donkey_golf.data_things.game_scoreboard - INFO - Determining where ties exist in the donkey scoreboard
2019-05-17 08:34:19,705 - donkey_golf.data_things.game_scoreboard - INFO - Adding ties to scoreboard data


In [23]:
df_sb = GS.data

In [4]:
loader = DT.LoadLeaderboard()

2019-05-16 13:52:02,942 - donkey_golf.data_things.base_class - DEBUG - ABOUT TO SET yaml_sql_dict
2019-05-16 13:52:02,953 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-16 13:52:03,230 - donkey_golf.data_things.base_class - INFO - Current tourney ID: 401056552
2019-05-16 13:52:03,231 - donkey_golf.data_things.base_class - INFO - Executing run_sql


In [5]:
loader.run()

2019-05-16 13:52:03,992 - donkey_golf.data_things.tourney_leaderboard - INFO - Tourney current status: in_progress
2019-05-16 13:52:03,993 - donkey_golf.data_things.tourney_leaderboard - INFO - Running clean_data function
2019-05-16 13:52:03,993 - donkey_golf.data_things.tourney_leaderboard - INFO - Running prepare_in_progress_data
2019-05-16 13:52:03,995 - donkey_golf.data_things.tourney_leaderboard - INFO - Calculating the donkey_score column
2019-05-16 13:52:04,007 - donkey_golf.data_things.tourney_leaderboard - INFO - Determining score for cut players and adding missed_cut indicator
2019-05-16 13:52:04,016 - donkey_golf.data_things.tourney_leaderboard - INFO - Determine if the player is active and if he has been cut or not
2019-05-16 13:52:04,035 - donkey_golf.data_things.base_class - INFO - Executing run_sql
2019-05-16 13:52:04,348 - donkey_golf.data_things.tourney_leaderboard - DEBUG - Adding golfer team count
2019-05-16 13:52:04,355 - donkey_golf.data_things.tourney_leaderboard 

In [ ]:
self.data.loc[self.data['today'] == '-', 'player_left'] = 0

In [ ]:
lb.pull_pre_tourney_data()

In [ ]:
lb.df_pre_tourney['current_rank'] = lb.df_pre_tourney['current_rank'].fillna(9999).astype(int).astype(str)

lb.df_pre_tourney['current_rank'] = lb.df_pre_tourney['current_rank'].replace('9999', '-')

In [ ]:
lb.df_pre_tourney

In [ ]:
dd = DT.DraftDay()

In [ ]:
dd.run()

In [ ]:
df = dd.data

In [ ]:
# top 8 pick 2
# next 12 pick 3
# 

In [ ]:
df

In [ ]:
df['rank'] = df['rank'].fillna(999)
df['rank'] = df['rank'].astype(int)
df['tier'] = 'Tier 3'
df.loc[df['rank'] <= 8, 'tier'] = 'Tier 1'
df.loc[(df['rank'] >= 9) & (df['rank'] <= 16), 'tier'] = 'Tier 2'

In [ ]:
df

In [ ]:
lb.df_pre_tourney.loc[lb.df_pre_tourney['current_rank'].notnull(), 'current_rank'] = lb.df_pre_tourney.loc[lb.df_pre_tourney['current_rank'].notnull(), 'current_rank'].astype(int)

In [ ]:
lb.df_pre_tourney.loc[lb.df_pre_tourney['current_rank'].notnull(), 'current_rank'].astype(int)

In [ ]:
lb.df_pre_tourney

In [ ]:
DT.LoadWorldRankings().run()

In [ ]:
DT.TournamentInfo().run()

In [ ]:
lb.run()

In [ ]:
lb.data

In [ ]:
lb.determine_current_round()

In [ ]:
lb.calculate_cut_line()

In [ ]:
df = lb.data

In [ ]:
self = lb

In [ ]:
cur_round = self.determine_current_round()

if cur_round == 'r2':
    sql = self.yaml_sql_dict.get('calculate_cut_line')
    df = self.run_sql(sql)
    # Grab the cut_str
    cut_str = df['pos'][0].lower()
    cut_dict = {}

    # Not sure what will happen if cut is even...
    if cut_str[-1:] == 'E':
        cut_dict['cut_str'] = 'E'
        cut_dict['cut_in'] = 0
    else:
        # Just grab the last two characters - this should be the cut score
        cut_str = cut_str[-2:]
        # Put both string and int representations into a dict
        cut_dict['cut_str'] = cut_str
        cut_dict['cut_int'] = int(cut_str.replace('+', '').replace('E', '0'))

    return cut_dict
else:
    logger.info("Dont need to worry about the cut line now")
    return None

In [ ]:
df

In [ ]:
print(sql)

In [ ]:
sql = self.yaml_sql_dict.get('calculate_cut_line')
df = self.run_sql(sql)
# Grab the cut_str
cut_str = df['pos'][0].lower()
cut_dict = {}

In [ ]:
user = DT.UserData(user_id=6)

In [ ]:
user.run()

In [ ]:
user.data

In [ ]:
gs = DT.GameScoreboard()

In [ ]:
gs.run()

In [ ]:
gs.data

In [ ]:
wha = DT.TeamPlayerResults()

In [ ]:
wha.run()

In [ ]:
wha.data.sort_values('username')

In [ ]:
lb = DT.LoadLeaderboard()

In [ ]:
lb.run()

In [ ]:
gs = dg.data_things.GameScoreboard()

In [ ]:
int(0)

In [ ]:
gs.tourney_status

In [ ]:
gs.run()

In [ ]:
cmon = dg.data_things.LoadLeaderboard()

In [ ]:
cmon.run()

In [ ]:
lb = dg.data_things.LoadLeaderboard()

In [ ]:
lb.tourney_status

In [ ]:
lb.scrape_espn_leaderboard(lb.url_lb)

In [ ]:
lb.data

In [ ]:
lb.determine_tourney_status()

In [ ]:
lb.load_tourney_status()

In [ ]:
vamp.run()

In [ ]:
vamp.tourney_status

In [ ]:
new = dg.data_things.PullLeaderboard(user_id=10)

In [ ]:
new.tourney_status

In [ ]:
new.run()

In [ ]:
new.pull_tourney_leaderboard()

In [ ]:
new.user_df

In [ ]:
new.user_id

In [ ]:
new.pull_users_team()

In [ ]:
new.data

In [ ]:
new.pull_tourney_leaderboard()

In [ ]:
getattr(new, 'user_df')

In [ ]:
print(getsource(new.pull_tourney_leaderboard))

In [ ]:
new.user_df

In [ ]:
pd.merge(new.data, new.user_df,
        how='outer',
        on='player')

In [ ]:
new.tourney_status

In [ ]:
gs = dg.data_things.GameScoreboard()

In [ ]:
gs.tourney_status

In [ ]:
gs.run()

In [ ]:
wha = dg.data_things.LoadWorldRankings()

In [ ]:
wha = dg.data_things.TournamentInfo()

In [ ]:
wha.run()

In [ ]:
draft = dg.data_things.DraftDay()

In [ ]:
draft.run()

In [ ]:
df = draft.data

In [ ]:
for col in ['current_rank','events_played','rank']:
    self.data.loc[self.data[col].notnull(), col] = self.data.loc[self.data[col].notnull(), col].astype(int).astype(str)

In [ ]:
df

In [ ]:
draft.data['current_rank']

In [ ]:
vamp.new_determine_status()['value'][0].strip()

In [ ]:
vamp.run()

In [ ]:
vamp.load_tourney_status()

In [ ]:
vamp.load_status_sql

In [ ]:
with psycopg2.connect(vamp.conf.db_url) as conn:
    cur = conn.cursor()
    cur.execute("delete from donkey_variables where metric = 'current_tourney_status';")
    cur.execute("INSERT INTO donkey_variables VALUES ('current_tourney_status', 'pre_tourney');")
    #cur.close()

In [ ]:
conn = psycopg2.connect(host=pg_credential.hostname,
                        port=pg_credential.port,
                        user=pg_credential.username,
                        password=pg_credential.password,
                        database=pg_credential.path[1:]) # To remove slash

cursor = conn.cursor()
cursor.execute("INSERT INTO a_table (c1, c2, c3) VALUES(%s, %s, %s)", (v1, v2, v3))

In [ ]:
getattr(cmon, 'tourney_status')

In [ ]:
vamp.load_tourney_status()

In [ ]:
vamp.tourney_status

In [ ]:
pd.read_sql("INSERT INTO donkey_variables VALUES ('current_tourney_status', 'pre_tourney');")

In [ ]:
cmon = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
cmon.run()

In [ ]:
cmon.tourney_status

In [ ]:
cmon.pull_tourney_leaderboard()

In [ ]:
if cmon.data.empty:
    print("EMPTY")

In [ ]:
cmon.tourney_status

In [ ]:
getattr(cmon, 'tourney_status')

In [ ]:
getattr(cmon, 'tourney_statusss', 'whoops')

In [ ]:
sb = dg.data_things.GameScoreboard()

In [ ]:
sb.run()

In [ ]:
vamp.golfer_count_by_team

In [ ]:
df = vamp.data

In [ ]:
df.head(2)

In [ ]:
df.loc[df['team_count']> 0]

In [ ]:
huh = dg.data_things.PullLeaderboard(user_id=6)

In [ ]:
huh.run()

In [ ]:
huh.data['team_count'].value_counts()

In [ ]:
df_tourney = huh.data
df_tourney = df_tourney.loc[df_tourney['team_count'] > 0]

In [ ]:
from datetime import datetime

In [ ]:
datetime.today()

In [ ]:
df_tourney

In [ ]:
ada = dg.data_things.PullLeaderboard(user_id=5)

In [ ]:
ada.pull_tourney_leaderboard()

In [ ]:
ada.data.team_count.value_counts()

In [ ]:
ada.calc_refresh_date()

In [ ]:
ada.determine_tourney_status()

In [ ]:
ada.pull_users_team()

In [ ]:
ada.user_df

In [ ]:
ada.sql_user